<a href="https://colab.research.google.com/github/qkrqhdud/GMB_Gemma2-/blob/main/Gemma-2-9B_PPP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
%pip install -U transformers
%pip install -U datasets
%pip install -U accelerate
%pip install -U peft
%pip install -U trl
%pip install -U bitsandbytes
%pip install -U wandb
%pip install flash-attn
%pip install -U bitsandbytes accelerate
%pip install unsloth

In [2]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"


In [3]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch, wandb
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format
import bitsandbytes as bnb
import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

In [4]:
from huggingface_hub import login

login(token='')

wb_token = ''

wandb.login(key=wb_token)
run = wandb.init(
    project='Fine-tune Gemma-2-9b-it',
    job_type="training",
    anonymous="allow"
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dbsgh5579 (dbsgh5579-lg-cns). Use `wandb login --relogin` to force relogin


In [5]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

In [6]:
base_model = "google/gemma-2-9b-it"
dataset_name = ""
new_model = "Gemma-2-9b-it-ppp"

In [7]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-9b",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.9.post3: Fast Gemma2 patching. Transformers = 4.45.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/6.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

In [32]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma2ForCausalLM(
      (model): Gemma2Model(
        (embed_tokens): Embedding(256000, 3584)
        (layers): ModuleList(
          (0-41): 42 x Gemma2DecoderLayer(
            (self_attn): Gemma2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3584, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3584, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(
  

In [8]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.9.post3 patched 42 layers with 42 QKV layers, 42 O layers and 42 MLP layers.


In [9]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

alpaca_data_cleaned.json:   0%|          | 0.00/44.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

In [10]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/51760 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [11]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 51,760 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 54,018,048
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
1,1.692300
2,2.194000
3,1.564200
4,1.714900
5,1.361300
6,1.317500
7,0.936600
8,1.101600
9,0.923500
10,1.013400


In [15]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import spacy # NLP
import itertools as it # Transform an object into an iterator. This is especially useful for generator objects
import re # Used with re.sub method, allowing us to remove patterns from a string
from gensim.models import Phrases # Trained model returning Sentences/Phrases, which are group of words generally associated together, like "ice_cream"
from gensim.corpora import Dictionary # Create a Dictionnary for LDA
from gensim.models.ldamulticore import LdaModel #Latent Dirichlet Allocation
from gensim.models import CoherenceModel # Calculate the coherence of our LDA model, which will tells us if the model is good or not

In [16]:
data = pd.read_pickle('/content/drive/MyDrive/PPP/cleaned_data.pkl')

In [17]:
data["content"][0]

'Robert Lerman  writes that achieving a healthy future of work requires employees to build skills that help them attain productive and rewarding careers. He notes - "one of the most cost-effective ways to do this is through apprenticeship, which helps workers master occupations and gain professional identity and pride". Coudlnt agree more!  #workbasedlearning   #USA   #apprenticeship   Read the article on  #UrbanWire   Urban Institute \n \n \n …see more'

In [57]:
from transformers import GenerationConfig

In [14]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<bos>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nContinue the fibonnaci sequence.\n\n### Input:\n1, 1, 2, 3, 5, 8\n\n### Response:\n13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6']

In [36]:
post_prompt = """Please rate the following blog post from 1 to 10 on each of the following category: Topic Relevance, Value Provision, Storytelling, and Expertise.
Please also provide a brief comment explaining your score for each criterion.

### Instruction:  Evaluate and Comment the blog post below and only provide the response.


### Input:
{}

### Response example:

"category": "Topic Relevance",
  "score": 8,
  "explanation": "The post is relevant to LinkedIn's professional environment, focusing on company growth, leadership, and recruitment, which are commonly discussed topics in the business world.",
  "improvement_suggestion": "To enhance relevance, the post could explicitly connect these insights to broader industry trends or professional development topics, making it more universally appealing."

  "category": "Value Provision",
  "score": 7,
  "explanation": "The post provides insights into company leadership and growth, but it is somewhat vague about specific strategies or lessons that would be valuable to other professionals.",
  "improvement_suggestion": "Incorporate more specific examples or takeaways that could benefit professionals in similar roles or industries, such as detailed growth strategies or leadership insights."

  "category": "Storytelling",
  "score": 6,
  "explanation": "The post mentions company growth and team collaboration but lacks a personal or compelling story that would engage readers on a deeper level.",
  "improvement_suggestion": "Introduce a short anecdote or personal reflection on a challenge or success to enhance the storytelling element and create a stronger connection with readers."

  "category": "Expertise",
  "score": 7,
  "explanation": "The author's knowledge is evident in their discussion of business growth and leadership, but the post could benefit from more detailed examples to showcase deeper expertise.",
  "improvement_suggestion": "Provide more concrete examples or metrics of the company's success, or delve into specific leadership strategies that highlight the author's expertise."



### Response:
{}"""

In [39]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    post_prompt.format(
        data["content"][0], # input
        ""# output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, use_cache = True)
tokenizer.batch_decode(outputs)



NameError: name 'generated_text' is not defined

In [44]:
generated_text=tokenizer.batch_decode(outputs)

In [45]:
generated_text

['<bos>Please rate the following blog post from 1 to 10 on each of the following category: Topic Relevance, Value Provision, Storytelling, and Expertise. \nPlease also provide a brief comment explaining your score for each criterion.\n\n### Instruction:  Evaluate and Comment the blog post below and only provide the response.\n\n\n### Input:\nRobert Lerman  writes that achieving a healthy future of work requires employees to build skills that help them attain productive and rewarding careers. He notes - "one of the most cost-effective ways to do this is through apprenticeship, which helps workers master occupations and gain professional identity and pride". Coudlnt agree more!  #workbasedlearning   #USA   #apprenticeship   Read the article on  #UrbanWire   Urban Institute \n \n \n …see more\n\n### Response example:\n\n"category": "Topic Relevance",\n  "score": 8,\n  "explanation": "The post is relevant to LinkedIn\'s professional environment, focusing on company growth, leadership, and 

In [47]:
response_start = generated_text[0].find("### Response:")
if response_start != -1:
  response = generated_text[0][response_start + len("### Response:"):].strip()
  print(response)

"category": "Topic Relevance",
  "score": 8,
  "explanation": "The post is relevant to LinkedIn's professional environment, focusing on the importance of skills development and the role of apprenticeships in achieving a healthy future of work.",
  "improvement_suggestion": "To enhance relevance, the post could provide more specific examples of how apprenticeships can help workers master occupations and gain professional identity and pride."

  "category": "Value Provision",
  "score": 8,
  "explanation": "The post provides valuable insights into the benefits of apprenticeships for workers and the role they can play in achieving a healthy future of work.",
  "improvement_suggestion": "To further enhance the value, the post could provide more detailed information on how apprenticeships can help workers build skills and achieve productive and rewarding careers."

  "category": "Storytelling",
  "score": 7,
  "explanation": "The post effectively communicates the importance of apprenticeshi